In [1]:
import tkinter as tk
import sounddevice as sd
from scipy.io.wavfile import write
import threading
import datetime
import numpy as np
import whisper
import os

In [2]:
# Global variables
fs = 44100  # Sample rate
recording = []
is_recording = False

In [3]:
def start_recording():
    global recording, is_recording
    is_recording = True
    recording = []

    def callback(indata, frames, time, status):
        if is_recording:
            recording.append(indata.copy())

    stream = sd.InputStream(samplerate=fs, channels=1, callback=callback)
    stream.start()
    start_button.config(state="disabled")
    stop_button.config(state="normal")
    output_label.config(text="🎤 Recording...")

    def keep_stream_open():
        while is_recording:
            sd.sleep(100)
        stream.stop()
        stream.close()

    threading.Thread(target=keep_stream_open).start()


In [4]:
def stop_recording():
    global is_recording
    is_recording = False
    start_button.config(state="normal")
    stop_button.config(state="disabled")
    output_label.config(text="🛑 Recording stopped...")

    # Save to WAV
    audio = np.concatenate(recording, axis=0)
    filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
    write(filename, fs, audio)
    output_label.config(text=f"✅ Saved as {filename}\n🔍 Transcribing...")

    # Transcribe using Whisper
    model = whisper.load_model("base")  # You can also use "small", "medium", etc.
    result = model.transcribe(filename)

    # Show transcribed text
    transcription = result["text"]
    output_label.config(text=f"📝 Transcription:\n{transcription}")

In [5]:
# GUI setup
root = tk.Tk()
root.title("🎙️ Voice-to-Text Recorder")
root.geometry("400x300")

start_button = tk.Button(root, text="Start Recording", command=start_recording, bg="green", fg="white", font=('Arial', 12))
start_button.pack(pady=10)

stop_button = tk.Button(root, text="Stop Recording", command=stop_recording, bg="red", fg="white", font=('Arial', 12))
stop_button.pack(pady=10)
stop_button.config(state="disabled")

output_label = tk.Label(root, text="", wraplength=380, justify="left", font=('Arial', 10))
output_label.pack(pady=20)

root.mainloop()

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
